# Machine Learning - Assignment 3

## Artificial Neural Network

The aim of the assignment is to implement an artificial neural network (mostly) from scratch. This includes implementing or fixing the following:

* Add support for additional activation functions and their derivatives.
* Add support for loss functions and their derivative.
* Add the use of a bias in the forward propagation.
* Add the use of a bias in the backward propagation.

In addition, you will we doing the following as well:

* Test the algorithm on 3 datasets.
* Compare neural networks with and without scaling.
* Hyper-parameter tuning.

The forward and backward propagation is made to work through a single layer, and are re-used multiple times to work for multiple layers.

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the implementation.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, you might find yourself going back and and forth to change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries.

**IMPORTANT NOTE:** You may not import any more libraries than the ones already imported!

In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# We set seed to better reproduce results later on.
np.random.seed(12345)

## Neural Network utility functions

### 1) Activation functions

Below is some setup for choosing activation function. Implement 2 additional activation functions, "ReLU" and one more of your choosing.

In [113]:
# Activation functions
def activate(activations, selected_function = "none"):
    
    if selected_function == "none":
        y = activations
    elif selected_function == "relu" :
        y=np.maximum(0,activations)
    elif selected_function == "sigmoid" :
        y=1/(1+np.exp((-activations)))
    return y

In [114]:
test_data = np.array([-2, -1, 0, 1, 2])
print("Sigmoid results",activate(test_data,selected_function="sigmoid"))
print("Relu results",activate(test_data,selected_function="relu"))

Sigmoid results [0.11920292 0.26894142 0.5        0.73105858 0.88079708]
Relu results [0 0 0 1 2]


### 2) Activation function derivatives

Neural networks need both the activation function and its derivative. Finish the code below.

In [115]:
def d_activate(activations, selected_function = "none"):
    if selected_function == "none":
        dy = np.ones_like(activations)
    elif selected_function == "relu":
        dy=np.where(activations>0,1,0)
    elif selected_function == "sigmoid":
        y=activate(activations,selected_function="sigmoid")
        dy=y*(1-y)
    return dy

In [116]:
test_data = np.array([-2, -1, 0, 1, 2])
print("Sigmoid results",d_activate(test_data,selected_function="sigmoid"))
print("Relu results",d_activate(test_data,selected_function="relu"))

Sigmoid results [0.10499359 0.19661193 0.25       0.19661193 0.10499359]
Relu results [0 0 0 1 1]


### 3) Loss functions

To penalize the network when it predicts incorrect, we need to meassure how "bad" the prediction is. This is done with loss-functions.

Similar as with the activation functions, the loss function needs its derivative as well.

Finish the MSE_loss (Mean Squared Error loss), as well as adding one additional loss function.

In [117]:
# This is the loss for a set of predictions y_hat compared to a set of real valyes y
def MSE_loss(y_hat, y):
    square_error=(y_hat-y)**2
    loss = np.mean(square_error)
    return loss

#Binary Cross-Entropy because we use Sigmoid Function above for activation
def Binary_CE(y_hat, y):
    epsilon=1e-15
    y_hat=np.clip(y_hat,epsilon,1-epsilon)
    loss = -np.mean(y*np.log(y_hat)+(1-y)*np.log(1-y_hat))
    return loss

The derivatives of the loss is with respect to the predicted value **y_hat**.

In [118]:
def d_MSE_loss(y_hat, y):
    dy = 2 * (y_hat - y) / y.size
    return dy

#Binary Cross-Entropy because we use Sigmoid Function above for activation
def d_Binary_CE(y_hat, y):
    epsilon=1e-15
    y_hat=np.clip(y_hat,epsilon,1-epsilon)
    dy = -(y/y_hat - (1-y)/(1-y_hat)) / y.size
    return dy

In [119]:
y_hat = np.array([0.9, 0.2, 0.8])
y     = np.array([1.0, 0.0, 1.0])
print(d_MSE_loss(y_hat, y))
print(d_Binary_CE(y_hat, y))

[-0.06666667  0.13333333 -0.13333333]
[-0.37037037  0.41666667 -0.41666667]


### 4) Forward propagation

The first "fundamental" function for neural networks is to be able to propagate the data forward through the neural network. We will implement this function here.

In [120]:
def propagate_forward(weights, activations, bias, activation_function="none"):
    
    # TODO: Add support for the use of bias

    dot_product = np.dot(weights, activations) + bias
    new_activations = activate(dot_product, activation_function)
    return new_activations

In [121]:
weights = np.array([[2, -1]])   # shape (1,2)
activations = np.array([3, 4])  # input
bias = np.array([1])  
print(bias)
out = propagate_forward(weights, activations, bias, activation_function="none")
print(out)          # bias

[1]
[3]


### 5) Back-propagation

To be able to train a neural network, we need to be able to propagate the loss backwards and update the weights. We will implement this function here.

In [122]:
# Calculates the backward gradients that are passed throught the layer in the backward pass.
# Returns both the derivative of the loss in respect to the weights and the input signal (activations).

def propagate_backward(weights, activations, dl_dz, bias, activation_function="none"):
    # NOTE: dl_dz is the derivative of the loss based on the previous layers activations/outputs

    # TODO: Add support for the use of bias

    dot_product = np.dot(activations, weights) + bias

    d_loss = d_activate(dot_product, activation_function) * dl_dz
    d_weights = np.dot(activations.T, d_loss)
    d_bias = np.sum(d_loss, axis=0, keepdims=True)
    d_activations = np.dot(d_loss, weights.T)
    
    return d_weights,d_bias, d_activations

In [126]:

activations = np.array([[1, 2]])     # shape (1,2)
weights = np.array([[2,3],
                    [4,5]])
bias        = np.array([[0, 0]])     # shape (1,2)
dl_dz       = np.array([[1, 1]])     # shape (1,2)

dW, db, dA = propagate_backward(weights, activations, dl_dz, bias, activation_function="none")

print("dW =", dW)
print("db =", db)
print("dA =", dA)

dW = [[1 1]
 [2 2]]
db = [[1 1]]
dA = [[5 9]]


## Neural network implementation

### 6) Fixing the neural network

Below is a class implementation of a MLP neural network. This implementation is still lacking several areas that are needed for the network to be robust and function well. Your task is to improve and fix it with the following:

1. Add a bias to the activation functions, and make sure the bias is also updated during training. 
2. Add a function that trains the network using minibatches (such that the neural network trains on a few samples at a time). 
3. Make use of an validation set in the training function. The model should stop training when the loss starts to increase for the validatin set. This feature should be able to be turned on and off to test the difference.


In [124]:
class NeuralNet(object):
    
    # Setup all parameters and activation functions.
    # This function runs directly when a new instance of this class is created.
    def __init__ (self, input_dim, output_dim, neurons = []):

        # NOTE: The "neurons" parameter is given as a list.
        # E.g., [4, 8, 4] means 4 neurons in layer 1, 8 neurons in layer 2 etc...

        # TODO: Add support for bias for each neuron in the code below.
        
        self.weights = [np.random.normal(0,2,(n,m)) for n,m in
                   zip([input_dim] + neurons, neurons + [output_dim])]
        
        self.activation_functions = ["relu"] * len(neurons) + ["none"]
    
    
    # Predict the input throught the network and calculate the output.
    def forward(self, x):

        # TODO: Add support for a bias for each neuron in the code below.
        for layer_weights, layer_activation_function in zip(self.weights, self.activation_functions):
            x = propagate_forward(layer_weights, x, layer_activation_function)
            
        return x
    
    
    # Adjust the weights in the network to better fit the desired output (y), given the input (x).
    # The weight updates are happening "in-place", thus we are only returning the loss from this function.
    # Note that this function can handle a variable size of the input (x), both full datasets or smaller parts of the dataset.
    def adjust_weights(self, x, y, learning_rate=1e-4):
                
        # TODO: Add support for a bias for each neuron and make sure these are learnt as well in the code below.

        activation = x
        activation_history = [] # NOTE: We need the previous (or intermediate) activations to make use of the "chain rule" (see lecture notes).
        
        for layer_weights, layer_activation_function in zip(self.weights, self.activation_functions):
            activation_history.append(activation)
            activation = propagate_forward(layer_weights, activation, layer_activation_function)
           
        # NOTE: The "activation" variable is changing as we go forward in the neural network.

        loss = MSE_loss(activation,y)
        d_activations = d_MSE_loss(activation,y) # NOTE: The final output can be "seen as" the final activations, thus the name.
        
        for layer_weights, layer_activation_function, previous_activations in reversed(list(zip(self.weights, self.activation_functions, activation_history))):

            d_weights, d_activations = propagate_backward(layer_weights, previous_activations, d_activations, layer_activation_function)

            # NOTE: Here is where the weight update is happening.
            layer_weights -= learning_rate * d_weights
                        
        return loss
    
    
    # A function for the training of the network.
    def train_net(self, x, y, batch_size=32, epochs=100, learning_rate=1e-4, use_validation_data=False):
        
        # TODO: Add a training loop where the weights and biases of the network is learnt over several epochs.

        # TODO: Add support for mini batches. That is, in each epoch the data should be split into several
        #       smaller subsets and the model should be trained on each of these subsets one at a time.

        # TODO: Implement the use of validation data, that is, splitting the training data into training data and validation data.
        #       The validation data should be used to stop the training when the model stops to generalise and starts to overfit.
        #       This feature should be able to be turned on and off to test the difference.

        # NOTE: Make use of previously implemented functions here.

        ...
    

## Train Neural Networks

### 7) Simple test

In this a very simple test for you to use and toy around with before using the datasets.

Make sure to test both the **adjust_weights** function and the **train_net** function. What is the difference between the two?

Also, be sure to **plot the loss for each epoch** to see how the network training is progressing!

In [125]:
# TODO: You can change most things in this cell if you want to, we encurage it!

n = 1000
d = 4

k = np.random.randint(0,10,(d,1))
x = np.random.normal(0,1,(n,d))
y = np.dot(x,k) + 0.1 + np.random.normal(0,0.01,(n,1))

                        
nn = NeuralNet(d, 1, [18, 12])

loss_1 = [nn.adjust_weights(x, y) for _ in range(1000)] 

loss_2 = train_net(...) # TODO: Use the train_net function to compare with the "adjust_weights" function.

plt.plot(loss_1)
plt.title("Loss 1")
plt.show()

plt.plot(loss_2)
plt.title("Loss 2")
plt.show()

ValueError: shapes (4,18) and (1000,4) not aligned: 18 (dim 1) != 1000 (dim 0)

### Real test and preprocessing

When using real data and neural networks, it is very important to scale the data between smaller values, usually between 0 and 1. This is because neural networks struggle with larger values as input compared to smaller values. 

To test this, we will use our first dataset and test with and without scaling.

Similar as with assignment 2, we will use the scikit-learn library for this preprocessing: https://scikit-learn.org/stable/modules/preprocessing.html

### 8) Dataset 1: Wine - with and without scaling

Read about the Wine dataset: https://archive.ics.uci.edu/dataset/109/wine

Train two neural network, one with scaling and one without. Are we able to see any difference in training results or loss over time?

**Note:** Do not train for to many epochs (more than maybe 50-100). The network might "learn" anyway in the end, but you should still be able to see a difference when training.

In [ ]:
from sklearn import preprocessing # For one-hot-encoding and scaling if needed
from sklearn.model_selection import train_test_split

data_wine = pd.read_csv("wine.csv").to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your neural networks.
# NOTE: Use the same train/test split for both neural network models!

# TODO: Do the above at least 3 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One combined lineplot with the scaling and one without the scaling, 2 plots in total.
# NOTE: Plot both the accuracy and the loss!

## Real data and hyper-parameter tuning

Now we are going to use real data, preprocess it, and do hyper-parameter tuning.

Choose two hyper-parameters to tune to try and achive an even better result. **Each chosen parameter needs to have at least 3 different values**. For example, if we were to choose "epochs" then we could test 10, 50, and 100 (optionally more). 

To better see what each hyper-parameter does to the models performance, we recommend doing a *grid-search* on the two chosen parameters. A grid-search tests how the two parameters interact with eachother. There might be a setting in parameter 1 that in combination with a setting in parameter 2 makes the model perform significantly better, compared to the other combinations. A grid-search test all these combinations.

**IMPORTANT NOTE:** Changing the **number of epochs** can be part of the tuning, but it **does not count** towards the two hyper parameters you choose.

### 9) Dataset 2: Mushroom

Read about the Mushroom dataset: https://archive.ics.uci.edu/dataset/73/mushroom

In [ ]:
from sklearn import preprocessing # For one-hot-encoding and scaling if needed
from sklearn.model_selection import train_test_split

data_mushroom = pd.read_csv("mushroom.csv").to_numpy()

# TODO: Preprocess the data.
# NOTE: You can choose how to handle data with missing values. Either remove or "fix" the missing data.

# TODO: Split the data into train and test

# TODO: Train a neural network on the data

# TODO: Visualize the loss for each epoch

# TODO: Visulaize the test accuracy for each epoch

When hyper-parameter tuning, please write the parameters and network sizes you test here:

* Parameter 1: 
* Parameter 2:

* Neural network sizes: 

In [ ]:
# TODO: Hyper-parameter tuning

# TODO: Visualize the loss after hyper-parameter tuning for each epoch

# TODO: Visulaize the test accuracy after hyper-parameter tuning for each epoch

### 10) Dataset 3: Adult

Read about the Adult dataset: https://archive.ics.uci.edu/dataset/2/adult

**IMPORTANT NOTE:** This dataset is much larger than the previous two (48843 instances). If your code runs slow on your own computer, either run this on the compute-server, or you may exclude parts of this dataset. But you must keep a minimum of 10000 datapoints.

In [ ]:
from sklearn import preprocessing # For one-hot-encoding and scaling if needed
from sklearn.model_selection import train_test_split

dataset_3 = pd.read(...) # TODO: Read the data.

# TODO: Preprocess the data.
# NOTE: You can choose how to handle data with missing values. Either remove or "fix" the missing data.

# TODO: Split the data into train and test

# TODO: Train a neural network on the data

# TODO: Visualize the loss for each epoch

# TODO: Visulaize the test accuracy for each epoch

When hyper-parameter tuning, please write the parameters and network sizes you test here:

* Parameter 1: 
* Parameter 2:

* Neural network sizes: 

In [ ]:
# TODO: Hyper-parameter tuning

# TODO: Visualize the loss after hyper-parameter tuning for each epoch

# TODO: Visulaize the test accuracy after hyper-parameter tuning for each epoch

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why would we want to use different activation functions?

2) Why would we want to use different loss functions?

3) Why are neural networks sensitive to large input values?

4) What is the role of the bias? 

5) What is the purpose of hyper-parameter tuning?

# Finished!

Was part of the setup incorrect? Did you spot any inconsistencies in the assignment? Could something improve?

If so, please write them and send via email and send it to:

* marcus.gullstrand@ju.se

Thank you!